# Парсинг данных с сайта **avito.ru**

In [1]:
# импорт необходимых модулей для работы парсера
from bs4 import BeautifulSoup
from tqdm import tqdm
from time import sleep
import requests
import random
from urllib.parse import quote
import os
import re
import pandas as pd

## Подготовка

Для того, чтобы сайт не мог так просто заподозрить, что мы берем данные с него, возьмем заголовки с реального браузера.

In [2]:
# достаем список юзер-агентов в список user_agents
with open('user_agents.txt', 'r') as parse_items:
    user_agents = list(map(lambda x: x.strip(), parse_items.readlines()))


Указываем ссылку, с которой будем брать данные `data_url` и ссылку-якорь, которая позволит создавать ссылки на страницы с объявлениями `base_url`.

In [3]:
# для москвы
data_url = 'https://www.avito.ru/moskva/kvartiry/prodam-ASgBAgICAUSSA8YQ'
# для всех регионов
# data_url = 'https://www.avito.ru/all/kvartiry/prodam-ASgBAgICAUSSA8YQ'

base_url = 'https://www.avito.ru'

# Очень желательно поставить реальную куку с сайта, иначе очень быстро забанят
cookie = кука

# не рекомендуется создавать данные больше 4000, так как реальных объявлений квартир не так много и со временем они начинают повторяться
dataset_size = 3000
links = set()

# создаем папки для сохранения данных
folder_to_save_csvs = 'final_csvs'
folder_to_save_htmls = 'final_html_pages'


## Приступаем к делу (парсим ссылки на объявления)

Для начала нам придется достать все ссылки с объявлениями

In [4]:
# проверяем, что папки для сохранения данных существуют, если нет, то создаем
os.path.exists(folder_to_save_htmls) or os.makedirs(folder_to_save_htmls)
os.path.exists(folder_to_save_csvs) or os.makedirs(folder_to_save_csvs)
print('Созданы папки для сохранения данных')


Созданы папки для сохранения данных


In [5]:
# добавляем pbar для отслеживания прогресса
pbar = tqdm(total=dataset_size, desc='Links obtained')

current_page = 1
while len(links) < dataset_size:

    # запрашиваем страницу с объявлениями
    page = requests.get(
        data_url,
        # для получения следующей страницы нужно передать параметр p с номером страницы, которую мы будем получать
        params={'p': current_page},
        headers={
            'User-Agent': quote(random.choice(user_agents)),

            # Очень желательно поставить реальную куку с сайта, иначе очень быстро забанят
            'Cookie': cookie,
        }
    )

    # кладем страницу в BeautifulSoup, чтобы парсить ее
    soup = BeautifulSoup(page.text, 'html.parser')
    prev_len = len(links)

    # парсим страницу и достаем ссылки на объявления
    for item in soup.find_all(name='a', attrs={'data-marker': 'item-title'}):
        if str(item.get('href')).startswith('http'):
            continue

        # Получаем ссылку на одно из объявлений
        cur_link = f'{base_url}{item.get("href")}'.strip()

        # Если эта ссылка уже была, пропускаем
        if cur_link in links:
            continue
        
        # сохраняем страницу с объявлением
        links.add(cur_link)

        # если достигли нужного количества объявлений, то выходим из цикла
        if len(links) >= dataset_size:
            break
    
    # обновляем pbar
    pbar.update(len(links) - prev_len)
    current_page += 1

    # задержка между запросами, чтобы не забанили
    sleep(2.5 + random.uniform(0, 1))

# удаляем pbar из памяти
del pbar

Links obtained: 100%|██████████| 3000/3000 [05:40<00:00,  8.81it/s]


In [6]:
# преобразуем links из set в list
links = list(links)


После того, как все ссылки получены, записываем на в файл, для того, чтобы нечаянно не потерять их в будущем

In [7]:
print("Примеры ссылок:")
print(*links[:5], sep='\n')
print()
print("Всего ссылок {}".format(len(links)))

# сохраняем ссылки в файл
with open(f'{folder_to_save_csvs}/links.csv', 'w') as links_csv:
    links_csv.write('\n'.join(links))


Примеры ссылок:
https://www.avito.ru/moskva/kvartiry/2-k._kvartira_51m_1125et._2480816191
https://www.avito.ru/moskva/kvartiry/kvartira-studiya_135m_19et._2969332239
https://www.avito.ru/moskva/kvartiry/2-k._kvartira_653m_78et._3034523105
https://www.avito.ru/moskva/kvartiry/4-k._kvartira_80m_45et._2604792981
https://www.avito.ru/moskva/kvartiry/kvartira-studiya_132m_45et._2967624312

Всего ссылок 3000


## Достаем объявления

После того, как мы получили все ссылки на объявления, приступим к парсингу самих объявлений

Небольшая фукнция, которая проверяет наличие необходимых тегов на странице

In [1]:
# проверяем, что в html-страницах есть нужные нам данные
def parse_items(x: str):
    if x is None:
        return False
    return 'params-paramsList__item' in x \
        or 'style-item-params-list-item' in x
    # or 'style-item-address__string' in x \
    # or 'style-item-map-wrapper' in x and 'style-expanded' in x


Скачиваем страницы **аналогично** тому, как мы скачивали ссылки на объявления, только теперь проверяем на валидность и сохраняем файлы.

Необходимость в скачивании нужна по нескольким причинам. Во-первых, для того, чтобы изменить таблицу с данными, не нужно заново качать страницы, достаточно изменить код их обрабатывающий. Во-вторых данные на сайте могут устареть: ранее работавшая ссылка может измениться, могут измениться данные по ссылке (иногда ссылка может просто перестать работать).

In [9]:
pbar = tqdm(total=len(links), desc='Links obtained')

for index, link in enumerate(links):
    res = requests.get(
        url=link,
        headers={
            'User-Agent': quote(random.choice(user_agents)),

            # Очень желательно поставить реальную куку с сайта, иначе очень быстро забанят
            'Cookie': cookie,
        }
    )

    soup = BeautifulSoup(res.text)

    parsed_data = soup.find_all(
        name='li',
        class_=lambda x: parse_items(x)
    )

    if len(parsed_data) == 0:
        print(f'Что то пошло не так со ссылкой {index}\n{link}')
        with open(f'{folder_to_save_csvs}/trouble_links.txt', 'a') as trouble_links:
            trouble_links.write(link + '\n')

        continue

    filename = f"{folder_to_save_htmls}/{link.split('/')[-1]}.html"
    with open(filename, 'w') as html_file:
        html_file.write(res.text)

    with open(f'{folder_to_save_csvs}/data.csv', 'a') as output_csv:
        output_csv.write(f'{link},{filename}\n')

    pbar.update()
    sleep(2.5 + random.uniform(0, 1))

Links obtained:   3%|▎         | 77/3000 [04:41<2:48:42,  3.46s/it]

Что то пошло не так со ссылкой 77
https://www.avito.ru/moskva/kvartiry/kvartira-studiya_19m_116et._3035086882


Links obtained:  47%|████▋     | 1409/3000 [1:29:17<1:45:50,  3.99s/it]

Что то пошло не так со ссылкой 1410
https://www.avito.ru/moskva/kvartiry/6-k._kvartira_1582m_1213et._2652014141


Links obtained:  69%|██████▊   | 2058/3000 [2:12:22<1:01:24,  3.91s/it]

Что то пошло не так со ссылкой 2060
https://www.avito.ru/moskva/kvartiry/kvartira-studiya_223m_1214et._2936990921


Links obtained:  86%|████████▌ | 2583/3000 [2:47:41<29:13,  4.21s/it]  

Что то пошло не так со ссылкой 2586
https://www.avito.ru/moskva/kvartiry/1-k._kvartira_40m_59et._2889683365


Links obtained: 100%|█████████▉| 2996/3000 [3:16:47<00:16,  4.11s/it]

## Парсим страницы

Теперь, когда у нас есть все необходимое, можно начать работать с данными.

Напишем функцию, превращающую html страничку в словарь с данными

In [10]:
# Напишем функцию, превращающую html страничку в словарь с данными
def parse_html(filename: str):
    # открываем файл с html
    with open(filename) as html_file:
        soup = BeautifulSoup(html_file.read())

    # достаем из html нужные нам данные
    parsed_data = soup.find_all(
        name='li',
        class_=lambda x: parse_items(x)
    )

    # если не нашли ни одного элемента, то возвращаем None
    if not len(parsed_data):
        return None

    # удаляем из html теги <style></style>, чтобы они не мешались при парсинге
    parsed_data[0] = re.sub('<style.*<\/style>', '', str(parsed_data[0]))

    # парсим оставшиеся данные
    price = float(soup.find(name='span', attrs={
                  'itemprop': 'price'}).attrs['content'])
    currency = soup.find(name='span', attrs={
                         'itemprop': 'priceCurrency'}).attrs['content']

    location = soup.find(
        name='span', class_=lambda x: False if x is None else 'style-item-address__string-' in x)

    parsed_data = {x.contents[0].contents[0]: x.contents[1]
                   for x in parsed_data[1:]}
    parsed_data['Цена'] = price
    parsed_data['Валюта'] = currency
    if len(location.contents) < 1:
        parsed_data['Местоположение'] = None
    else:
        parsed_data['Местоположение'] = location.contents[0]

    return parsed_data

# пример работы функции
parse_html(
    f'{folder_to_save_htmls}/{links[0].split("/")[-1]}.html')

{'Общая площадь': '51\xa0м²',
 'Площадь кухни': '14.2\xa0м²',
 'Этаж': '11 из 25',
 'Тип комнат': 'изолированные',
 'Санузел': 'совмещенный',
 'Ремонт': 'дизайнерский',
 'Способ продажи': 'свободная',
 'Тип дома': 'монолитный',
 'Этажей в доме': '25',
 'Двор': 'закрытая территория, детская площадка, спортивная площадка',
 'Парковка': 'подземная',
 'Цена': 22000000.0,
 'Валюта': 'RUB',
 'Местоположение': 'Москва, Береговой пр., 5Ак4'}

Получим подобные словари для всех наших данных

In [11]:
all_dicts = []
with open(f'{folder_to_save_csvs}/data.csv') as data_csv:
    for line in data_csv.readlines():
        link, filename = line.strip().split(',')

        cur_dict = parse_html(filename)
        if cur_dict is None:
            continue

        cur_dict['Ссылка'] = link

        all_dicts.append(cur_dict)

for dict_ in all_dicts[:5]:
    print(dict_)

{'Общая площадь': '51\xa0м²', 'Площадь кухни': '14.2\xa0м²', 'Этаж': '11 из 25', 'Тип комнат': 'изолированные', 'Санузел': 'совмещенный', 'Ремонт': 'дизайнерский', 'Способ продажи': 'свободная', 'Тип дома': 'монолитный', 'Этажей в доме': '25', 'Двор': 'закрытая территория, детская площадка, спортивная площадка', 'Парковка': 'подземная', 'Цена': 22000000.0, 'Валюта': 'RUB', 'Местоположение': 'Москва, Береговой пр., 5Ак4', 'Ссылка': 'https://www.avito.ru/moskva/kvartiry/2-k._kvartira_51m_1125et._2480816191'}
{'Общая площадь': '13.5\xa0м²', 'Этаж': '1 из 9', 'Высота потолков': '2.7\xa0м', 'Санузел': 'совмещенный', 'Окна': 'во двор', 'Ремонт': 'требует ремонта', 'Способ продажи': 'свободная', 'Вид сделки': 'возможна ипотека', 'Тип дома': 'кирпичный', 'Год постройки': '1974', 'Этажей в доме': '9', 'В доме': 'мусоропровод', 'Двор': 'детская площадка', 'Парковка': 'открытая во дворе', 'Цена': 3500000.0, 'Валюта': 'RUB', 'Местоположение': 'Москва, ул. Фабрициуса, 8', 'Ссылка': 'https://www.avi

## Сохраняем данные

In [12]:
df = pd.DataFrame(all_dicts)
df.head()

,Общая площадь,Площадь кухни,Этаж,Тип комнат,Санузел,Ремонт,Способ продажи,Тип дома,Этажей в доме,Двор,...,Балкон или лоджия,Техника,Тёплый пол,Отделка,Название новостройки,Официальный застройщик,Тип участия,Срок сдачи,"Корпус, строение",Запланирован снос
0,51 м²,14.2 м²,11 из 25,изолированные,совмещенный,дизайнерский,свободная,монолитный,25,"закрытая территория, детская площадка, спортив...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13.5 м²,NaN,1 из 9,NaN,совмещенный,требует ремонта,свободная,кирпичный,9,детская площадка,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,65.3 м²,9.2 м²,7 из 8,изолированные,раздельный,косметический,свободная,панельный,8,"детская площадка, спортивная площадка",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,80 м²,8.3 м²,4 из 5,смежные,раздельный,требует ремонта,альтернативная,кирпичный,5,"детская площадка, спортивная площадка",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13.2 м²,NaN,4 из 5,NaN,совмещенный,евро,свободная,кирпичный,5,"детская площадка, спортивная площадка",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Т.к. в датафрейме содержатся запятые, изменим `sep`.

In [13]:
df.to_csv(f'{folder_to_save_csvs}/flats.csv', index=False, sep='^')